In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences

# print(os.listdir("../input"))

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

In [3]:
df = pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [4]:
df.drop('source', axis=1, inplace=True)

In [5]:
df.isnull().sum()

english_sentence    2
hindi_sentence      0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
def data_cleaning(text):

    # Convert to lower
    text = text.lower()

    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove digits
    text = re.sub(r"\d", '', text)
    
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text)
    
    return text

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127605 entries, 0 to 127606
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   english_sentence  127605 non-null  object
 1   hindi_sentence    127605 non-null  object
dtypes: object(2)
memory usage: 2.9+ MB


In [9]:
df['cleaned_english_sentence'] = df['english_sentence'].apply(data_cleaning)
df['cleaned_hindi_sentence'] = df['hindi_sentence'].apply(data_cleaning)

In [10]:
# Add start and end tokens to target sequences
df['cleaned_hindi_sentence'] = df['cleaned_hindi_sentence'].apply(lambda x : '<START> '+ x + ' <END>')

In [11]:
# Shuffle the dataframe
df = df.sample(n=1000, random_state=42).reset_index(drop=True)

In [12]:
df.head()

,english_sentence,hindi_sentence,cleaned_english_sentence,cleaned_hindi_sentence
0,He declares the result and reports it to the E...,वही परिणाम की घोषणा करता है और निर्वाचन आयोग क...,he declares the result and reports it to the e...,<START> वही परिणाम की घोषणा करता है और निर्वाच...
1,was a little uncomfortable for them.,थोडा कठिन था।,was a little uncomfortable for them,<START> थोडा कठिन था। <END>
2,"A multi-purpose auditorium , a branch of the S...","बहुउद्देशीय सभागार , भारतीय स्टेट बैंक की शाखा...",a multipurpose auditorium a branch of the stat...,<START> बहुउद्देशीय सभागार भारतीय स्टेट बैंक क...
3,No fees is to be paid for filing the appeal to...,अधिकरण में अपील करने के लिए कोई फीस नहीं देनी ...,no fees is to be paid for filing the appeal to...,<START> अधिकरण में अपील करने के लिए कोई फीस नह...
4,headind kaun banega crorepati,शीर्षक कौन बनेगा करोड़पति (Kaun Banega Crorepa...,headind kaun banega crorepati,<START> शीर्षक कौन बनेगा करोड़पति kaun banega ...


In [13]:
df['hindi_sentence'][0], df['cleaned_hindi_sentence'][0]

('वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है .',
 '<START> वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है  <END>')

In [14]:
#Tokenize the texts and convert to sequences
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='')
en_tokenizer.fit_on_texts(df['cleaned_english_sentence'])
en_sequences = en_tokenizer.texts_to_sequences(df['cleaned_english_sentence'])

hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='')
hi_tokenizer.fit_on_texts(df['cleaned_hindi_sentence'])
hi_sequences = hi_tokenizer.texts_to_sequences(df['cleaned_hindi_sentence'])

english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(hi_tokenizer.word_index) + 1
print("English Vocab Size: ", english_vocab_size)
print("Hindi Vocab Size: ", hindi_vocab_size)

English Vocab Size:  4414
Hindi Vocab Size:  5080


In [15]:
en_max_length = df['cleaned_english_sentence'].apply(lambda x:len(str(x).split())).max()
hi_max_length = df['cleaned_hindi_sentence'].apply(lambda x:len(str(x).split())).max()

en_max_length, hi_max_length

(146, 165)

In [16]:
#Prepare encoder data
encoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(en_sequences, maxlen=en_max_length, padding='post')

In [17]:
#Prepare decoder data
decoder_inputs = []
decoder_outputs = []

for hi in hi_sequences:
  decoder_inputs.append(hi[:-1])
  decoder_outputs.append(hi[1:])

decoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs, maxlen=hi_max_length, padding='post')
decoder_outputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs, maxlen=hi_max_length, padding='post')

In [18]:
# encoder_inputs

In [19]:
# decoder_inputs, decoder_outputs

In [20]:
# df.shape[0], en_max_length, hi_max_length

In [39]:
# Training and Testing split
# 90%, 10%
split = int(0.90 * df.shape[0])
print(split)

X_train = [encoder_inputs[:split], decoder_inputs[:split]]
y_train = decoder_outputs[:split]

# Test data to evaluate our NMT model using BLEU score
X_test = [encoder_inputs[split:], decoder_inputs[split:]]
y_test = decoder_outputs[split:]

print(X_train[0].shape, X_train[1].shape, y_train.shape)
     
print(X_test[0].shape, X_test[1].shape, y_test.shape)

900
(900, 146) (900, 165) (900, 165)
(100, 146) (100, 165) (100, 165)


In [40]:
from tensorflow.keras.initializers import HeNormal
#Define LSTM model
d_model = 256

#Encoder
inputs = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(english_vocab_size, d_model, mask_zero=True)(inputs)
_,state_h,state_c = tf.keras.layers.LSTM(d_model,activation='tanh',return_state=True)(x)

#Decoder
targets = tf.keras.layers.Input(shape=(None,))
embedding_layer = tf.keras.layers.Embedding(hindi_vocab_size, d_model, mask_zero=True)
x = embedding_layer(targets)
decoder_lstm = tf.keras.layers.LSTM(d_model,activation='tanh',return_sequences=True, return_state=True)
x,_,_ = decoder_lstm(x, initial_state=[state_h, state_c])
dense1 = tf.keras.layers.Dense(hindi_vocab_size, activation='tanh')
x = dense1(x)

model = tf.keras.models.Model(inputs=[inputs, targets],outputs=x)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimize = tf.keras.optimizers.Adam(0.001)
model.compile(optimizer='adam', loss=loss)

model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 256) │  1,129,984 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, None)      │          0 │ input_layer_4[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 256) │  1,300,480 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    525,312 │ embedding_2[0][0… │
│                     │ (None, 256),      │            │ not_equal_3[0][0] │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    525,312 │ embedding_3[0][0… │
│                     │ 256), (None,      │            │ lstm_2[0][1],     │
│                     │ 256), (None,      │            │ lstm_2[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │  1,305,560 │ lstm_3[0][0]      │
│                     │ 5080)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,786,648 (18.26 MB)

 Trainable params: 4,786,648 (18.26 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))
     

Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - loss: 11.5477 - val_loss: 10.5068
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 9.9558 - val_loss: 10.3996
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - loss: 9.2974 - val_loss: 9.3773
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 8.2984 - val_loss: 8.6741
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - loss: 8.4769 - val_loss: 8.5500
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - loss: 8.4889 - val_loss: 8.5401
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 8.4851 - val_loss: 8.5411
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - loss: 8.4795 - val_loss: 8.5075
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - loss: 8.4282 - val_loss: 8.4743
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - loss: 8.4122 - val_loss: 8.4848
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - loss: 8.4355 - val_loss: 8.4826
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - loss: 

In [42]:
#Inference Model
#Encoder
encoder = tf.keras.models.Model(inputs, [state_h, state_c])

#Decoder
decoder_input_h = tf.keras.layers.Input(shape=(d_model,))
decoder_input_c = tf.keras.layers.Input(shape=(d_model,))
x = embedding_layer(targets)
x, decoder_output_h, decoder_output_c = decoder_lstm(x, initial_state=[decoder_input_h, decoder_input_c])
x = dense1(x)
decoder = tf.keras.models.Model([targets] + [decoder_input_h, decoder_input_c], 
                                [x] + [decoder_output_h, decoder_output_c])

In [43]:
decoder.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 256) │  1,300,480 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    525,312 │ embedding_3[1][0… │
│                     │ 256), (None,      │            │ input_layer_6[0]… │
│                     │ 256), (None,      │            │ input_layer_7[0]… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │  1,305,560 │ lstm_3[1][0]      │
│                     │ 5080)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,131,352 (11.95 MB)

 Trainable params: 3,131,352 (11.95 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# My indepth analysis about encoder and decoder outputs

# X_train[0][0]
a, b = encoder.predict(X_train[[0][0]])

29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 154ms/step


In [ ]:
# X_train[0][0]

In [ ]:
# a, b = encoder.predict(X_train[[0][0]])

In [ ]:
# X_train[0]

In [ ]:
# X_train[[0][0]]

In [ ]:
# X_train[[0][0]]

In [29]:
# original_texts = []
# X_test = X_train[0][0]

en_tokenizer.sequences_to_texts([X_train[0][0]])

# for sequence in X_test:
#     original_text = en_tokenizer.sequences_to_texts(sequence)[0]
#     original_texts.append(original_text)
    
# original_texts

['he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house                                                                                                                               ']

In [ ]:
# a[0]

In [30]:
target_seq = np.zeros(1)
target_seq[0] = hi_tokenizer.word_index['start']

In [ ]:
# target_seq.shape, a[0].reshape(1, -1).shape, b[0].shape

In [31]:
output_tokens, h, c = decoder.predict([target_seq.reshape(1, -1)] + [a[0].reshape(1, -1), b[0].reshape(1, -1)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [38]:
output_tokens[0][0]

array([-0.17923562, -0.12630852, -0.1736213 , ..., -0.06300774,
       -0.07561757, -0.05123287], dtype=float32)

In [ ]:
# h.shape, c.shape, output_tokens.shape

In [ ]:
# output_tokens[0][0].shape

In [32]:
np.argmax(output_tokens[0][0])

1126

In [33]:
stop_condition = False
decoded_sentence = ''
# state_values = encoder_h, encoder_c
# h = a[0]
# c = b[0]
while not stop_condition:
    output_tokens, h, c = decoder.predict([target_seq.reshape(1, -1)] + [h.reshape(1, -1), c.reshape(1, -1)])
#         output_tokens, h, c = decoder.predict([target_seq.reshape(1, -1)] + [state_values])
    print(output_tokens.shape)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_token_index = np.argmax(output_tokens[0][0])
    print(sampled_token_index)
#     sampled_char = reverse_target_char_index[sampled_token_index]
    sampled_char = hi_tokenizer.index_word[sampled_token_index]
    decoded_sentence += ' '+sampled_char

    # Exit condition: either hit max length
    # or find stop character.
    if (sampled_char == 'end' or
       len(decoded_sentence) > 50):
        stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros(1)
    target_seq[0] = sampled_token_index
    h = h
    c = c

#         states_value = [h, c]
# #     return decoded_sentence

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
(1, 1, 5080)
1126
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
(1, 1, 5080)
1126
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
(1, 1, 5080)
1356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
(1, 1, 5080)
1356


In [ ]:
# decoded_sentence

In [ ]:
# hi_tokenizer.index_word

In [44]:
def decode_sequence(input_seq):
    a, b = encoder.predict(input_seq)
    print(a.shape, b.shape)
    target_seq = np.zeros(1)
    target_seq[0] = hi_tokenizer.word_index['start']

    stop_condition = False
    decoded_sentence = ''
#     state_values = encoder_h, encoder_c
    h = a[0]
    c = b[0]
    # h = a[0]
    # c = b[0]
    while not stop_condition:
        output_tokens, h, c = decoder.predict([target_seq.reshape(1, -1)] + [h.reshape(1, -1), c.reshape(1, -1)])
#         output_tokens, h, c = decoder.predict([target_seq.reshape(1, -1)] + [state_values])
#         print(output_tokens.shape)

        # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token_index = np.argmax(output_tokens[0][0])
#         print(sampled_token_index)
    #     sampled_char = reverse_target_char_index[sampled_token_index]
        sampled_char = hi_tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'end' or
           len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros(1)
        target_seq[0] = sampled_token_index
        h = h
        c = c

#         states_value = [h, c]
    return decoded_sentence

In [ ]:
# X_train[0][0].reshape(1,-1)

In [ ]:
# y_train[0][0]

In [45]:
print(en_tokenizer.sequences_to_texts([X_train[0][0]]))
print(hi_tokenizer.sequences_to_texts([y_train[0]]))

decode_sequence(X_train[[0][0]])

['he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house                                                                                                                               ']
['वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है end                                                                                                                                                ']
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 136ms/step
(900, 256) (900, 256)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

' ताकतवर गायों नवीनकरण नवीनकरण नवीनकरण नवीनकरण मौके मौके मौके मौके मौके मौके मौके मौके मौके मौके मौके मौके'

In [ ]:
print(en_tokenizer.sequences_to_texts([X_test[0][0]]))
print(hi_tokenizer.sequences_to_texts([y_test[0]]))

decode_sequence(X_test[[0][0]])

In [ ]:
X_test.shape